# The Disaster 🤖
## Ahmed Ashraf & Abdelrahman Lotfy

# Load Dependencies 📦

In [ ]:
import torch
import torchvision.transforms as transforms
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
import torchvision
from tqdm import tqdm
import matplotlib.pyplot as plt
from IPython.display import clear_output
%matplotlib inline
import time

# Custom DataSet

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data_path, csv_file_path, transform=None):
        self.data = pd.read_csv(csv_file_path)
        self.transform = transform
        self.data_path = data_path

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        img_name = self.data_path + self.data.iloc[index, 0] 
        image = Image.open(img_name)
        label = self.data.iloc[index, 1]  

        if self.transform:
            image = self.transform(image)

        return image, label

# Importing The Data 💾


In [ ]:
csv_file_path = '/kaggle/input/ieeenu-cis-juniors-plant-binary-classification/PlantFinal Files-20230213T190104Z-001/Final Files/train.csv'
data_path = '/kaggle/input/ieeenu-cis-juniors-plant-binary-classification/PlantFinal Files-20230213T190104Z-001/Final Files/train/'


# data augmentation for training
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),    
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])


transform_test = transforms.Compose([
    transforms.Resize((224, 224)),    
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])


traindataset = CustomDataset(data_path,csv_file_path, transform=transform_train)
validation_size = int(0.2 * len(traindataset))
train_size = len(traindataset) - validation_size
train_subset, validation_subset = torch.utils.data.random_split(traindataset, [train_size, validation_size])
batch_size = 16
trainloader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=2)

# Data Visualization 🖼️

In [ ]:
def imshow(img, un):
    if(un):
        img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = next(dataiter)


# show images
imshow(torchvision.utils.make_grid(images), True)
imshow(torchvision.utils.make_grid(images), False)
print(images[0].shape)

# Class Distribution 📊

In [ ]:
label_counts = {}
for batch in trainloader:
    _, labels = batch  
    for label in labels:
        label = label.item()
        if label in label_counts:
            label_counts[label] += 1
        else:
            label_counts[label] = 1

label_names = list(label_counts.keys())
label_counts = list(label_counts.values())
plt.figure(figsize=(8, 8))
plt.pie(label_counts, labels=label_names, autopct='%1.1f%%', startangle=140)
plt.axis('equal')  
plt.title('Label Distribution in the Training Dataset')
plt.show()


# Balancing Classes ⚖️ 

In [ ]:
labels = [data_label for _, data_label in train_subset]
count_class_0 = labels.count(0)
count_class_1 = labels.count(1)
num_samples_to_keep = min(count_class_0, count_class_1
indices_class_0 = [i for i, label in enumerate(labels) if label == 0]
indices_class_1 = [i for i, label in enumerate(labels) if label == 1]
balanced_indices = np.random.choice(indices_class_1, size=num_samples_to_keep, replace=False)
balanced_indices = list(balanced_indices)
balanced_indices.extend(np.random.choice(indices_class_0, size=num_samples_to_keep, replace=False))
balanced_dataset = [train_subset[i] for i in balanced_indices]

# DataLoaders 🔧

In [ ]:
batch_size = 16
# Load the train dataset into a DataLoader
trainloader = DataLoader(balanced_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
# Load the validation dataset into a DataLoader
crossloader = DataLoader(validation_subset, batch_size=32, shuffle=False)

# Class Distribution After Balancing 📊

In [ ]:
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

label_counts = {}
for batch in trainloader:
    _, labels = batch  
    for label in labels:
        label = label.item()
        if label in label_counts:
            label_counts[label] += 1
        else:
            label_counts[label] = 1

label_names = list(label_counts.keys())
label_counts = list(label_counts.values())
plt.figure(figsize=(8, 8))
plt.pie(label_counts, labels=label_names, autopct='%1.1f%%', startangle=140)
plt.axis('equal')  
plt.title('Label Distribution in the Training Dataset')
plt.show()


In [ ]:
# Initialize counters
count_label_1 = 0
count_label_0 = 0

for batch_idx, (data, labels) in enumerate(trainloader):
    count_label_1 += (labels == 1).sum().item()
    count_label_0 += (labels == 0).sum().item()

# Print the counts
print("Count of labeled data with Label 1:", count_label_1)
print("Count of labeled data with Label 0:", count_label_0)


In [ ]:
import torch.nn as nn


# Model Architecture 👷

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models


densenet = models.densenet121(pretrained=True)
num_ftrs = densenet.classifier.in_features

# Replace the classifier with a custom fully connected layer followed by the Sigmoid activation
densenet.classifier = nn.Linear(num_ftrs, 2)


# Defining The Train Function 🚂


In [ ]:
def train(model, dataloader, loss_fn, optimizer, device, ldl, lts): ## ldl = length dataloader, lts = length dataset
    model.train()  ## puts the model on training mode such as enabling gradient computations
    total_loss = 0 ## over current epoch

    total_correct = 0 ## extra, calculate the accuracy on training set during epoch
    for batch in tqdm(dataloader):
        inputs, labels = batch[0].to(device), batch[1].to(device)
        optimizer.zero_grad() ## deletes stored gradients
        outputs = model(inputs)

        loss = loss_fn(outputs, labels)

        loss.backward()     ## computes gradients
        optimizer.step()    ## updates parameters

        total_loss += loss.item()

        predictions = outputs.argmax(dim=1)

        correct = (predictions == labels).sum().item()
        total_correct += correct

    return total_loss / ldl, total_correct / lts

# Accuracy Function 🎯


In [ ]:
def compute_accuracy(dataloader, model):
    model.eval()  # switch to evaluation mode

    total_correct = 0
    total_count = 0
    with torch.no_grad():
        for batch in dataloader:
            inputs, labels = batch[0].to(device), batch[1].to(device)
            outputs = model(inputs)
            predictions = outputs.argmax(dim=1)
            correct = (predictions == labels).sum().item()
            total_correct += correct
            total_count += len(labels)

    accuracy = total_correct / total_count

    return accuracy


# Model Initialization & Configuration ⚙️


In [ ]:
import torch.optim as optim

model = densenet ## initialize the model
# model= nn.DataParallel(model) ## incase you want to use the x2 gpu option you'd need to enable this

ldl = len(trainloader) ## fixed

bestScore = 0
patience = 6          # if model accuracy on validation dataset didn't improve for 10 epochs it will stop and save highest scoring model
num_epochs = 30
loss_fn = nn.CrossEntropyLoss()
best_epoch = 0

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
#optimizer = optim.Adam(model.parameters(), lr=0.001)
# scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)


## select the device that we will train the model on
if(torch.cuda.is_available()):
    device = torch.device('cuda')
elif(torch.backends.mps.is_available()): ## for the macbook users :)
    device = torch.device('mps')
else:
    device = torch.device('cpu')

model.to(device)

# Main Training Loop 🚂 🔁


In [ ]:
# the following lists will be used to plot the loss and accuracy curves by keeping track of the values over epochs
train_losses = []
cross_accs = []
train_accs = []
lr = []
for epoch in range(num_epochs):
    train_loss, train_acc = train(model, trainloader, loss_fn, optimizer, device, ldl, ldl*batch_size)
    CurrentScore = compute_accuracy(crossloader, model)

    lr.append(optimizer.param_groups[0]['lr'])
    train_losses.append(train_loss)
    cross_accs.append(CurrentScore)
    train_accs.append(train_acc)
    clear_output(wait=True) # wait for all plots to be shown, then erase them and display the updated ones

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, lr: {lr[-1]:.7f}')
    print(f'Cross Acc: {CurrentScore:.4f}, Train Acc: {train_acc:.4f}')
    # scheduler.step()

    # initialize 3 subplots to plot the loss curve, learning rate curve and accuracy curve
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    axs[0].plot(train_losses)
    axs[0].set_title('Training Loss')
    axs[1].plot(lr)
    axs[1].set_title('Learning Rate')
    axs[2].plot(cross_accs, label="cross")
    axs[2].plot(train_accs, label = "train")
    axs[2].set_title('Cross/Train Accuracy')
    axs[2].legend()
    plt.show(block=False)
    print("Patience REM : ", epoch - best_epoch)
    if CurrentScore > bestScore:
        bestScore = CurrentScore
        best_epoch = epoch
        torch.save(model.state_dict(), 'best_model.pt')
    elif epoch - best_epoch >= patience:
        print(f'Validation loss did not improve for {patience} epochs. Training stopped.')
        break

# Loading Back The Best Model 🔙



In [ ]:
model.load_state_dict(torch.load('/content/best_model.pt', map_location=device))

### Getting model Predictions 📝
we now will pass the best performing model state, into this function, which will return a list containing our labels

In [ ]:
def getPreds(dataloader, models):
    preds = []
    with torch.no_grad():
        for batch in dataloader:
            inputs = batch[0].to(device)
            outputs = model(inputs)
            predictions = outputs.argmax(dim=1)
            preds.extend(predictions.cpu().numpy())

    return preds

# Import Test Data 📥

In [ ]:
test_set = CustomDataset('/content/PlantFinal Files-20230213T190104Z-001/Final Files/test/','/content/PlantFinal Files-20230213T190104Z-001/Final Files/test.csv', transform=transform_test)
testloader = DataLoader(test_set, batch_size=32, shuffle=False)
test_data = pd.read_csv('/content/PlantFinal Files-20230213T190104Z-001/Final Files/test.csv')
filenames = test_data['Filename']

# Saving Submission Data 💽

In [ ]:
preds = getPreds(testloader, model)
submn = pd.DataFrame({'Filename' : filenames,'Label': preds})
submn.to_csv('subm.csv', index=False)
submn